# SAM3 特徵 PCA 分析

本筆記本可視化 SAM3 的特徵空間。它對從參考邊界框區域和目標圖像提取的特徵執行 PCA，以可視化它們的關係。


In [ ]:
import torch
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.cm as cm
import os
import sys
from sklearn.decomposition import PCA

# 確保我們可以導入 sam3
sys.path.append(os.getcwd())

from sam3 import sam3_model_registry, Sam3Processor
from sam3.model.data_misc import FindStage
from sam3.model import box_ops

# 配置
CHECKPOINT_PATH = os.path.abspath("sam3.pt")
MODEL_TYPE = "vit_l"

if torch.backends.mps.is_available():
    DEVICE = "mps"
elif torch.cuda.is_available():
    DEVICE = "cuda"
else:
    DEVICE = "cpu"

print(f"設備: {DEVICE}")


In [ ]:
# 載入 SAM3 模型
if not os.path.exists(CHECKPOINT_PATH):
    print(f"錯誤: 找不到 {CHECKPOINT_PATH}。")
else:
    print("正在載入模型...")
    sam3 = sam3_model_registry[MODEL_TYPE](checkpoint=CHECKPOINT_PATH)
    sam3.to(device=DEVICE)
    sam3.eval()
    processor = Sam3Processor(sam3, device=DEVICE)
    print("模型已載入！")


In [ ]:
# 載入圖像
# 請替換為您的實際圖像路徑
ref_path = "ref_demo.jpg" 
target_path = "target_demo.jpg" 

# 如果缺失則創建虛擬圖像（與這之前的腳本相同）
if not os.path.exists(ref_path):
    print("正在創建虛擬演示圖像...")
    ref_img = Image.new('RGB', (400, 400), color='black')
    from PIL import ImageDraw
    draw = ImageDraw.Draw(ref_img)
    draw.rectangle([100, 100, 200, 200], fill='red') 
    ref_img.save(ref_path)
    
    target_img = Image.new('RGB', (600, 600), color='black')
    draw = ImageDraw.Draw(target_img)
    draw.rectangle([50, 50, 150, 150], fill='red') 
    draw.rectangle([400, 100, 500, 200], fill='blue') 
    target_img.save(target_path)

ref_image = Image.open(ref_path).convert("RGB")
target_image = Image.open(target_path).convert("RGB")

# 參考框 [x0, y0, x1, y1]
REF_BOX = [100, 100, 200, 200]

plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.imshow(ref_image)
plt.title("參考圖")
ax = plt.gca()
w, h = REF_BOX[2]-REF_BOX[0], REF_BOX[3]-REF_BOX[1]
ax.add_patch(patches.Rectangle((REF_BOX[0], REF_BOX[1]), w, h, fill=False, edgecolor='green', linewidth=3))

plt.subplot(1, 2, 2)
plt.imshow(target_image)
plt.title("目標圖")
plt.show()


In [ ]:
# 提取特徵
print("正在執行骨幹網路推理...")
images = [ref_image, target_image]
state = processor.set_image_batch(images)

# 獲取特徵圖（最高層級）
# 形狀: [Batch, Channels, H_feat, W_feat]
features = state["backbone_out"]["backbone_fpn"][-1]
B, C, Hf, Wf = features.shape
print(f"特徵圖形狀: {features.shape}")

# 為 PCA 預處理特徵
# 1. 展平空間維度
# 2. 轉換為 [N, C]，其中 N 是批次中的總像素數（或僅目標）

# 讓我們專門獲取參考框特徵
ref_w, ref_h = ref_image.size
cx = (REF_BOX[0] + REF_BOX[2]) / 2 / ref_w
cy = (REF_BOX[1] + REF_BOX[3]) / 2 / ref_h
nw = (REF_BOX[2] - REF_BOX[0]) / ref_w
nh = (REF_BOX[3] - REF_BOX[1]) / ref_h

# 將標準化坐標轉換為特徵圖坐標
x0 = int((cx - nw/2) * Wf)
y0 = int((cy - nh/2) * Hf)
x1 = int((cx + nw/2) * Wf)
y1 = int((cy + nh/2) * Hf)
x0, y0 = max(0, x0), max(0, y0)
x1, y1 = min(Wf, x1), min(Hf, y1)

# 提取參考特徵
ref_feat_map = features[0] # [C, Hf, Wf]
ref_crop = ref_feat_map[:, y0:y1, x0:x1] # [C, h_crop, w_crop]
# 展平為向量 [N_ref, C]
ref_vectors = ref_crop.permute(1, 2, 0).reshape(-1, C)

# 提取目標特徵
target_feat_map = features[1] # [C, Hf, Wf]
target_vectors = target_feat_map.permute(1, 2, 0).reshape(-1, C)

print(f"參考向量: {ref_vectors.shape}")
print(f"目標向量: {target_vectors.shape}")


In [ ]:
# 執行 PCA
# 我們想要找到參考物件特徵的主成分
# 並驗證目標物件是否共享這些成分。

# 選項 A：僅在參考特徵上擬合 PCA，然後轉換目標。
# 選項 B：在目標特徵上擬合 PCA，查看參考是否聚類。
# 選項 C：在組合集上擬合 PCA。

# 讓我們嘗試選項 A：在參考上擬合。如果參考特徵是一致的，PC1 應該捕捉到參考的"物件性"。
# 檢查目標上的 PC1 映射應該會突出顯示相似的區域。

# 數據轉 numpy
X_ref = ref_vectors.detach().cpu().numpy()
X_target = target_vectors.detach().cpu().numpy()

# 正規化？
# 如果幅度變化顯著，對於深度特徵來說這通常是好的做法，
# 儘管 SAM 特徵通常已經過 LayerNorm。
X_ref = X_ref / np.linalg.norm(X_ref, axis=1, keepdims=True)
X_target = X_target / np.linalg.norm(X_target, axis=1, keepdims=True)

# 在參考上擬合 PCA
pca = PCA(n_components=3)
pca.fit(X_ref)

print(f"解釋變異比率 (參考擬合): {pca.explained_variance_ratio_}")

# 轉換目標
target_pca = pca.transform(X_target)
# 轉換參考 (查看其自身的投影)
ref_pca = pca.transform(X_ref)

# 重塑回圖像
target_pca_map = target_pca.reshape(Hf, Wf, 3)
ref_pca_map = ref_pca.reshape(y1-y0, x1-x0, 3) # 注意: 參考只是裁剪


In [ ]:
# 在目標圖像上可視化 PCA 嵌入
# 我們將 PC1, PC2, PC3 可視化為 RGB 通道 (或僅將 PC1 可視化為熱圖)

# 將成分歸一化到 0-1 以顯示 RGB
def normalize_pca(x):
    _min = x.min(axis=(0,1), keepdims=True)
    _max = x.max(axis=(0,1), keepdims=True)
    return (x - _min) / (_max - _min + 1e-6)

target_pca_vis = normalize_pca(target_pca_map)

# 放大到原始圖像大小
target_pca_img = Image.fromarray((target_pca_vis * 255).astype(np.uint8)).resize(target_image.size, Image.NEAREST)

plt.figure(figsize=(15, 6))
plt.subplot(1, 3, 1)
plt.title("目標圖像")
plt.imshow(target_image)
plt.axis('off')

plt.subplot(1, 3, 2)
plt.title("PCA 投影 (在參考上擬合)")
plt.imshow(target_pca_img)
plt.axis('off')

plt.subplot(1, 3, 3)
plt.title("PC1 熱圖 (與參考變異的相似性)")
plt.imshow(target_pca_map[:, :, 0], cmap='jet') # 僅 PC1
plt.axis('off')

plt.show()

print("解讀: PCA 投影中顏色相似的像素具有與參考物件相似的特徵。")


In [ ]:
# 替代方案：組合 PCA
# 在 參考 + 目標 上擬合 PCA 以可視化全局流形
X_combined = np.concatenate([X_ref, X_target], axis=0)
pca_global = PCA(n_components=3)
pca_global.fit(X_combined)

target_pca_global = pca_global.transform(X_target).reshape(Hf, Wf, 3)
target_pca_global_vis = normalize_pca(target_pca_global)
target_pca_global_img = Image.fromarray((target_pca_global_vis * 255).astype(np.uint8)).resize(target_image.size, Image.NEAREST)

plt.figure(figsize=(10, 6))
plt.title("全局 PCA (參考 + 目標)")
plt.imshow(target_pca_global_img)
plt.axis('off')
plt.show()
